In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data_tools.get_data import get_data
from src.assets.lumi import lumi_dict

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
era = '2016'
wdf, lumi = get_data(era, stitch_dy=True)

In [5]:
masses = wdf.mass.unique()
masses_df = pd.DataFrame([{"mass": mass} for mass in masses]).dropna().astype('int')

In [6]:
lumi = lumi_dict['16-18']

In [7]:
backgrounds = [
    {"name":'DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8',
     "xsec": 5929.0*1e3, "nevents":79575677, "nickname": "dy"} ,
    {"name" :'mc_ttbar', "xsec": 76.7*1e3, "nevents": 79140880, "nickname": "tt"}]
backgrounds = pd.DataFrame(backgrounds)

In [8]:
wdf.loc[wdf.name=='y3_400_deepflavour_bff', 'mass'] = 400
wdf.loc[wdf.name=='y3_1000_deepflavour_bff', 'mass'] = 1000 
wdf.loc[wdf.name=='y3_700_deepflavour_bff', 'mass'] = 700 
wdf.loc[wdf.name=='y3_250_deepflavour_bff', 'mass'] = 250

wdf.loc[wdf.name=='y3_400_deepflavour_bff', 'dbs'] = -1
wdf.loc[wdf.name=='y3_1000_deepflavour_bff', 'dbs'] = -1
wdf.loc[wdf.name=='y3_700_deepflavour_bff', 'dbs'] = -1
wdf.loc[wdf.name=='y3_250_deepflavour_bff', 'dbs'] = -1

In [10]:
signals = [
{"name" :"BFFZprimeToMuMu_M_250_dbs0p04", "xsec": 122.5, "nevents": 511129}, 
{"name" :"BFFZprimeToMuMu_M_300_dbs0p04", "xsec": 58.86, "nevents": 442196}, 
{"name" :"BFFZprimeToMuMu_M_400_dbs0p04", "xsec": 17.42, "nevents": 378564}, 
{"name" :"BFFZprimeToMuMu_M_500_dbs0p04", "xsec": 6.42, "nevents": 421806}, 
{"name" :"BFFZprimeToMuMu_M_500_dbs0p5", "xsec": 13.81, "nevents": 508321}, 
{"name" :"BFFZprimeToMuMu_M_500_dbs1p0", "xsec": 35.56, "nevents": 436956}, 
{"name" :"BFFZprimeToMuMu_M_350_dbs0p04", "xsec": 31.04, "nevents": 470696}, 
{"name" :"BFFZprimeToMuMu_M_350_dbs0p5", "xsec": 65.35, "nevents": 386677}, 
{"name" :"BFFZprimeToMuMu_M_350_dbs1p0", "xsec": 165.8, "nevents": 476490}, 
{"name" :"BFFZprimeToMuMu_M_200_dbs0p04", "xsec": 288.9, "nevents": 454692}, 
{"name" :"BFFZprimeToMuMu_M_200_dbs1p0", "xsec": 1486.0, "nevents": 507520}, 
{"name" :"BFFZprimeToMuMu_M_200_dbs0p5", "xsec": 592.6, "nevents": 479332}, 
{"name" :"BFFZprimeToMuMu_M_125_dbs0p04", "xsec": 1528.0, "nevents": 337381}, 
{"name" :"BFFZprimeToMuMu_M_125_dbs0p5", "xsec": 3119.0, "nevents": 347033}, 
{"name" :"BFFZprimeToMuMu_M_125_dbs1p0", "xsec": 7866.0, "nevents": 349518}, 
{"name" :"BFFZprimeToMuMu_M_150_dbs0p04", "xsec": 815.0, "nevents": 316360}, 
{"name" :"BFFZprimeToMuMu_M_150_dbs0p5", "xsec": 1673.0, "nevents": 327249}, 
{"name" :"BFFZprimeToMuMu_M_150_dbs1p0", "xsec": 4187.0, "nevents": 336801}, 
{"name" :"BFFZprimeToMuMu_M_175_dbs0p04", "xsec": 471.2, "nevents": 301727}, 
{"name" :"BFFZprimeToMuMu_M_175_dbs0p5", "xsec": 969.8, "nevents": 312508}, 
{"name" :"BFFZprimeToMuMu_M_175_dbs1p0", "xsec": 2438.0, "nevents": 319185}, 
{"name" :"BFFZprimeToMuMu_M_750_dbs0p04", "xsec": 0.9116, "nevents": 210705}, 
    
    {"name" :"y3_250_deepflavour_bff", "xsec": 10.1, "nevents": 435000}, 
    {"name" :"y3_400_deepflavour_bff", "xsec": 2.9, "nevents": 456000}, 
    
    {"name" :"y3_1000_deepflavour_bff", "xsec": .2, "nevents": 500000}, 
    
    {"name" :"y3_700_deepflavour_bff", "xsec": 0.6, "nevents": 469000}, 
    
]
signals = pd.DataFrame(signals)

In [11]:
signals['weight']  = lumi * signals['xsec'] / signals['nevents'] 
backgrounds['weight']  = lumi * backgrounds['xsec'] / backgrounds['nevents'] 

In [12]:
def df_mass_filter(df, mass, widthper):
    width = mass*widthper
    return df[ ((df.DiLepMass-mass) > -width) & ((df.DiLepMass-mass) < width)]

In [13]:
import uncertainties as unc

In [15]:
def region_counts(df, weight, postfix=""):
    sr1 = unc.ufloat(df[df.SR1_nom==1].shape[0]*weight, df[df.SR1_nom==1].shape[0]**.5*weight)
    sr2 = unc.ufloat(df[df.SR2_nom==1].shape[0]*weight, df[df.SR2_nom==1].shape[0]**.5*weight)
    return {"SR1{}".format(postfix): sr1, "SR2{}".format(postfix): sr2}

In [16]:
sample_counts = []
for i, signal in signals.iterrows():
    stdf = wdf[wdf.name==signal['name']]
    mass = stdf.mass.mean()
    stdf = df_mass_filter(stdf, mass, .1)
    signal_regions = region_counts(stdf, signal.weight)
    #get data stats
    bck_regions = {}
    for i, bck in backgrounds.iterrows():
        btdf = wdf[wdf.name==bck['name']]
        btdf = df_mass_filter(btdf, mass, .1)
        bck_regions = {**bck_regions, **region_counts(btdf, bck.weight, postfix=bck.nickname)}
    
    #fill sample list
    sample_dict = {"mass": int(mass), "dbs": round(stdf.dbs.mean(),2), **signal_regions, **bck_regions}
    sample_counts.append(sample_dict)

In [17]:
sample_counts_csv = pd.DataFrame(sample_counts)

In [18]:
sample_counts_csv.to_csv('sample_s_b.csv')

In [19]:
sample_counts_csv

,mass,dbs,SR1,SR2,SR1dy,SR2dy,SR1tt,SR2tt
0,250,0.04,1820+/-8,537+/-4,(1.53+/-0.13)e+03,(9.0+/-1.0)e+02,1237+/-13,1447+/-14
1,300,0.04,921+/-4,299.9+/-2.3,(8.6+/-0.9)e+02,(5.8+/-0.8)e+02,716+/-10,826+/-10
2,400,0.04,269.9+/-1.3,99.9+/-0.8,(3.1+/-0.6)e+02,(1.7+/-0.4)e+02,211+/-5,247+/-6
3,500,0.04,95.2+/-0.4,39.58+/-0.29,(2.5+/-0.5)e+02,103+/-32,71.3+/-3.1,78.0+/-3.2
4,500,0.50,161.9+/-0.8,65.2+/-0.5,(2.5+/-0.5)e+02,103+/-32,71.3+/-3.1,78.0+/-3.2
5,500,1.00,358.9+/-2.0,143.1+/-1.3,(2.5+/-0.5)e+02,103+/-32,71.3+/-3.1,78.0+/-3.2
6,350,0.04,491.5+/-2.1,171.7+/-1.2,(5.5+/-0.8)e+02,(3.8+/-0.6)e+02,394+/-7,444+/-8
7,350,0.50,820+/-4,277.2+/-2.5,(5.5+/-0.8)e+02,(3.8+/-0.6)e+02,394+/-7,444+/-8
8,350,1.00,1768+/-9,582+/-5,(5.5+/-0.8)e+02,(3.8+/-0.6)e+02,394+/-7,444+/-8
9,200,0.04,3634+/-18,959+/-9,(3.07+/-0.18)e+03,(1.52+/-0.12)e+03,1957+/-16,2243+/-17


In [20]:
sample_counts = pd.read_csv('sample_s_b.csv')

In [21]:
sample_counts = sample_counts_csv

In [22]:
sample_counts = sample_counts.loc[:, ~sample_counts.columns.str.contains('^Unnamed')].set_index(['mass', 'dbs']).sort_index()

In [23]:
sample_counts.sort_index().round(2)

SR1                SR2              SR1dy  \
mass dbs                                                                
125   0.04    (4.74+/-0.05)e+03          1150+/-27  (6.87+/-0.27)e+03   
      0.50    (7.68+/-0.10)e+03  (1.76+/-0.05)e+03  (6.87+/-0.27)e+03   
      1.00  (1.643+/-0.023)e+04  (3.69+/-0.11)e+03  (6.87+/-0.27)e+03   
150   0.04    (6.04+/-0.05)e+03          1370+/-22  (6.04+/-0.25)e+03   
      0.50    (9.57+/-0.08)e+03  (2.13+/-0.04)e+03  (6.04+/-0.25)e+03   
      1.00  (2.035+/-0.019)e+04  (4.37+/-0.09)e+03  (6.04+/-0.25)e+03   
175   0.04            4938+/-33          1200+/-16  (4.44+/-0.21)e+03   
      0.50    (8.00+/-0.06)e+03          1934+/-29  (4.44+/-0.21)e+03   
      1.00  (1.706+/-0.013)e+04  (3.95+/-0.06)e+03  (4.44+/-0.21)e+03   
200   0.04            3634+/-18            959+/-9  (3.07+/-0.18)e+03   
      0.50            5878+/-32          1508+/-16  (3.07+/-0.18)e+03   
      1.00  (1.262+/-0.007)e+04          3098+/-35  (3.07+/-0.18)e+03   
250  -1.00          165.8+/-0.7         98.5+/-0.6  (1.53+/-0.13)e+03   
      0.04             1820+/-8            537+/-4  (1.53+/-0.13)e+03   
300   0.04              921+/-4        299.9+/-2.3    (8.6+/-0.9)e+02   
350   0.04          491.5+/-2.1        171.7+/-1.2    (5.5+/-0.8)e+02   
      0.50              820+/-4        277.2+/-2.5    (5.5+/-0.8)e+02   
      1.00             1768+/-9            582+/-5    (5.5+/-0.8)e+02   
400  -1.00         46.26+/-0.20       31.78+/-0.17    (3.1+/-0.6)e+02   
      0.04          269.9+/-1.3         99.9+/-0.8    (3.1+/-0.6)e+02   
500   0.04           95.2+/-0.4       39.58+/-0.29    (2.5+/-0.5)e+02   
      0.50          161.9+/-0.8         65.2+/-0.5    (2.5+/-0.5)e+02   
      1.00          358.9+/-2.0        143.1+/-1.3    (2.5+/-0.5)e+02   
700  -1.00          7.41+/-0.04      6.088+/-0.033            41+/-21   
750   0.04         11.20+/-0.08        5.84+/-0.06            62+/-25   
1000 -1.00        1.853+/-0.010      1.778+/-0.010            0.0+/-0   

                        SR2dy        SR1tt       SR2tt  
mass dbs                                                
125   0.04  (3.56+/-0.19)e+03    1437+/-14   1756+/-15  
      0.50  (3.56+/-0.19)e+03    1437+/-14   1756+/-15  
      1.00  (3.56+/-0.19)e+03    1437+/-14   1756+/-15  
150   0.04  (2.54+/-0.16)e+03    2081+/-17   2436+/-18  
      0.50  (2.54+/-0.16)e+03    2081+/-17   2436+/-18  
      1.00  (2.54+/-0.16)e+03    2081+/-17   2436+/-18  
175   0.04  (2.22+/-0.15)e+03    2191+/-17   2502+/-18  
      0.50  (2.22+/-0.15)e+03    2191+/-17   2502+/-18  
      1.00  (2.22+/-0.15)e+03    2191+/-17   2502+/-18  
200   0.04  (1.52+/-0.12)e+03    1957+/-16   2243+/-17  
      0.50  (1.52+/-0.12)e+03    1957+/-16   2243+/-17  
      1.00  (1.52+/-0.12)e+03    1957+/-16   2243+/-17  
250  -1.00    (9.0+/-1.0)e+02    1237+/-13   1447+/-14  
      0.04    (9.0+/-1.0)e+02    1237+/-13   1447+/-14  
300   0.04    (5.8+/-0.8)e+02     716+/-10    826+/-10  
350   0.04    (3.8+/-0.6)e+02      394+/-7     444+/-8  
      0.50    (3.8+/-0.6)e+02      394+/-7     444+/-8  
      1.00    (3.8+/-0.6)e+02      394+/-7     444+/-8  
400  -1.00    (1.7+/-0.4)e+02      211+/-5     247+/-6  
      0.04    (1.7+/-0.4)e+02      211+/-5     247+/-6  
500   0.04           103+/-32   71.3+/-3.1  78.0+/-3.2  
      0.50           103+/-32   71.3+/-3.1  78.0+/-3.2  
      1.00           103+/-32   71.3+/-3.1  78.0+/-3.2  
700  -1.00            72+/-27    9.6+/-1.1  13.2+/-1.3  
750   0.04            62+/-25    5.9+/-0.9   7.3+/-1.0  
1000 -1.00            21+/-14  0.40+/-0.23   1.2+/-0.4

In [24]:
print(sample_counts.sort_index().round(2).to_latex())

\begin{tabular}{llllllll}
\toprule
     &       &                  SR1 &                SR2 &              SR1dy &              SR2dy &        SR1tt &       SR2tt \\
mass & dbs &                      &                    &                    &                    &              &             \\
\midrule
125  &  0.04 &    (4.74+/-0.05)e+03 &          1150+/-27 &  (6.87+/-0.27)e+03 &  (3.56+/-0.19)e+03 &    1437+/-14 &   1756+/-15 \\
     &  0.50 &    (7.68+/-0.10)e+03 &  (1.76+/-0.05)e+03 &  (6.87+/-0.27)e+03 &  (3.56+/-0.19)e+03 &    1437+/-14 &   1756+/-15 \\
     &  1.00 &  (1.643+/-0.023)e+04 &  (3.69+/-0.11)e+03 &  (6.87+/-0.27)e+03 &  (3.56+/-0.19)e+03 &    1437+/-14 &   1756+/-15 \\
150  &  0.04 &    (6.04+/-0.05)e+03 &          1370+/-22 &  (6.04+/-0.25)e+03 &  (2.54+/-0.16)e+03 &    2081+/-17 &   2436+/-18 \\
     &  0.50 &    (9.57+/-0.08)e+03 &  (2.13+/-0.04)e+03 &  (6.04+/-0.25)e+03 &  (2.54+/-0.16)e+03 &    2081+/-17 &   2436+/-18 \\
     &  1.00 &  (2.035+/-0.019)e+04 &  (4

In [25]:
sample_counts['s'] = sample_counts['SR1']
sample_counts['b'] = sample_counts['SR1dy'] + sample_counts['SR1tt']

In [26]:
def make_s_b(df):
    df['s/b'] = df.s/df.b
    df['s/sqrt(b)'] = df.s/df.b**.5
    df['s/sqrt(s+b)'] = df.s/(df.s+df.b)**.5
    return df

In [27]:
make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2)

s                  b              s/b  \
mass dbs                                                              
125   0.04    (4.74+/-0.05)e+03  (8.31+/-0.27)e+03    0.571+/-0.019   
      0.50    (7.68+/-0.10)e+03  (8.31+/-0.27)e+03    0.924+/-0.032   
      1.00  (1.643+/-0.023)e+04  (8.31+/-0.27)e+03      1.98+/-0.07   
150   0.04    (6.04+/-0.05)e+03  (8.12+/-0.25)e+03    0.744+/-0.024   
      0.50    (9.57+/-0.08)e+03  (8.12+/-0.25)e+03      1.18+/-0.04   
      1.00  (2.035+/-0.019)e+04  (8.12+/-0.25)e+03      2.51+/-0.08   
175   0.04            4938+/-33  (6.63+/-0.21)e+03    0.745+/-0.025   
      0.50    (8.00+/-0.06)e+03  (6.63+/-0.21)e+03      1.21+/-0.04   
      1.00  (1.706+/-0.013)e+04  (6.63+/-0.21)e+03      2.57+/-0.09   
200   0.04            3634+/-18  (5.02+/-0.18)e+03    0.724+/-0.026   
      0.50            5878+/-32  (5.02+/-0.18)e+03      1.17+/-0.04   
      1.00  (1.262+/-0.007)e+04  (5.02+/-0.18)e+03      2.51+/-0.09   
250  -1.00          165.8+/-0.7  (2.76+/-0.13)e+03  0.0600+/-0.0027   
      0.04             1820+/-8  (2.76+/-0.13)e+03    0.658+/-0.030   
300   0.04              921+/-4  (1.58+/-0.09)e+03    0.584+/-0.035   
350   0.04          491.5+/-2.1    (9.5+/-0.8)e+02      0.52+/-0.04   
      0.50              820+/-4    (9.5+/-0.8)e+02      0.87+/-0.07   
      1.00             1768+/-9    (9.5+/-0.8)e+02      1.87+/-0.15   
400  -1.00         46.26+/-0.20    (5.2+/-0.6)e+02    0.089+/-0.010   
      0.04          269.9+/-1.3    (5.2+/-0.6)e+02      0.52+/-0.06   
500   0.04           95.2+/-0.4    (3.2+/-0.5)e+02      0.30+/-0.05   
      0.50          161.9+/-0.8    (3.2+/-0.5)e+02      0.51+/-0.08   
      1.00          358.9+/-2.0    (3.2+/-0.5)e+02      1.13+/-0.18   
700  -1.00          7.41+/-0.04            51+/-21      0.15+/-0.06   
750   0.04         11.20+/-0.08            67+/-25      0.17+/-0.06   
1000 -1.00        1.853+/-0.010        0.40+/-0.23        4.6+/-2.7   

              s/sqrt(b)   s/sqrt(s+b)  
mass dbs                               
125   0.04   52.0+/-1.0    41.5+/-0.6  
      0.50   84.2+/-1.7    60.7+/-0.8  
      1.00      180+/-4   104.4+/-1.1  
150   0.04   67.0+/-1.2    50.8+/-0.5  
      0.50  106.2+/-1.9    71.9+/-0.7  
      1.00      226+/-4   120.6+/-0.9  
175   0.04   60.6+/-1.1    45.9+/-0.5  
      0.50   98.2+/-1.7    66.1+/-0.6  
      1.00      210+/-4   110.9+/-0.7  
200   0.04   51.3+/-0.9    39.1+/-0.4  
      0.50   82.9+/-1.5    56.3+/-0.5  
      1.00  178.1+/-3.3    95.0+/-0.6  
250  -1.00  3.15+/-0.07   3.06+/-0.07  
      0.04   34.6+/-0.8    26.9+/-0.4  
300   0.04   23.2+/-0.7  18.43+/-0.35  
350   0.04   16.0+/-0.6  12.96+/-0.34  
      0.50   26.6+/-1.1    19.5+/-0.4  
      1.00   57.4+/-2.3    33.9+/-0.5  
400  -1.00  2.03+/-0.11   1.95+/-0.10  
      0.04   11.8+/-0.6   9.61+/-0.35  
500   0.04    5.3+/-0.4   4.69+/-0.29  
      0.50    9.1+/-0.7     7.4+/-0.4  
      1.00   20.1+/-1.6    13.8+/-0.5  
700  -1.00  1.04+/-0.21   0.97+/-0.17  
750   0.04  1.36+/-0.25   1.26+/-0.20  
1000 -1.00    2.9+/-0.8   1.23+/-0.06

In [28]:
print(make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2).to_latex())

\begin{tabular}{lllllll}
\toprule
     &       &                    s &                  b &              s/b &    s/sqrt(b) &   s/sqrt(s+b) \\
mass & dbs &                      &                    &                  &              &               \\
\midrule
125  &  0.04 &    (4.74+/-0.05)e+03 &  (8.31+/-0.27)e+03 &    0.571+/-0.019 &   52.0+/-1.0 &    41.5+/-0.6 \\
     &  0.50 &    (7.68+/-0.10)e+03 &  (8.31+/-0.27)e+03 &    0.924+/-0.032 &   84.2+/-1.7 &    60.7+/-0.8 \\
     &  1.00 &  (1.643+/-0.023)e+04 &  (8.31+/-0.27)e+03 &      1.98+/-0.07 &      180+/-4 &   104.4+/-1.1 \\
150  &  0.04 &    (6.04+/-0.05)e+03 &  (8.12+/-0.25)e+03 &    0.744+/-0.024 &   67.0+/-1.2 &    50.8+/-0.5 \\
     &  0.50 &    (9.57+/-0.08)e+03 &  (8.12+/-0.25)e+03 &      1.18+/-0.04 &  106.2+/-1.9 &    71.9+/-0.7 \\
     &  1.00 &  (2.035+/-0.019)e+04 &  (8.12+/-0.25)e+03 &      2.51+/-0.08 &      226+/-4 &   120.6+/-0.9 \\
175  &  0.04 &            4938+/-33 &  (6.63+/-0.21)e+03 &    0.745+/-0.025 &  

In [29]:
sample_counts['s'] = sample_counts['SR2']
sample_counts['b'] = sample_counts['SR2dy'] + sample_counts['SR2tt']
make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2)

s                  b              s/b  \
mass dbs                                                            
125   0.04          1150+/-27  (5.31+/-0.19)e+03    0.216+/-0.009   
      0.50  (1.76+/-0.05)e+03  (5.31+/-0.19)e+03    0.331+/-0.015   
      1.00  (3.69+/-0.11)e+03  (5.31+/-0.19)e+03    0.694+/-0.032   
150   0.04          1370+/-22  (4.98+/-0.16)e+03    0.275+/-0.010   
      0.50  (2.13+/-0.04)e+03  (4.98+/-0.16)e+03    0.428+/-0.016   
      1.00  (4.37+/-0.09)e+03  (4.98+/-0.16)e+03    0.877+/-0.033   
175   0.04          1200+/-16  (4.73+/-0.15)e+03    0.254+/-0.009   
      0.50          1934+/-29  (4.73+/-0.15)e+03    0.409+/-0.015   
      1.00  (3.95+/-0.06)e+03  (4.73+/-0.15)e+03    0.835+/-0.030   
200   0.04            959+/-9  (3.76+/-0.13)e+03    0.255+/-0.009   
      0.50          1508+/-16  (3.76+/-0.13)e+03    0.401+/-0.014   
      1.00          3098+/-35  (3.76+/-0.13)e+03    0.824+/-0.029   
250  -1.00         98.5+/-0.6  (2.35+/-0.10)e+03  0.0419+/-0.0018   
      0.04            537+/-4  (2.35+/-0.10)e+03    0.229+/-0.010   
300   0.04        299.9+/-2.3  (1.41+/-0.08)e+03    0.213+/-0.012   
350   0.04        171.7+/-1.2    (8.2+/-0.6)e+02    0.209+/-0.016   
      0.50        277.2+/-2.5    (8.2+/-0.6)e+02    0.337+/-0.026   
      1.00            582+/-5    (8.2+/-0.6)e+02      0.71+/-0.05   
400  -1.00       31.78+/-0.17    (4.2+/-0.4)e+02    0.075+/-0.008   
      0.04         99.9+/-0.8    (4.2+/-0.4)e+02    0.237+/-0.024   
500   0.04       39.58+/-0.29           181+/-33      0.22+/-0.04   
      0.50         65.2+/-0.5           181+/-33      0.36+/-0.07   
      1.00        143.1+/-1.3           181+/-33      0.79+/-0.14   
700  -1.00      6.088+/-0.033            85+/-27    0.072+/-0.023   
750   0.04        5.84+/-0.06            69+/-25    0.085+/-0.031   
1000 -1.00      1.778+/-0.010            22+/-15      0.08+/-0.05   

               s/sqrt(b)   s/sqrt(s+b)  
mass dbs                                
125   0.04    15.8+/-0.5    14.3+/-0.4  
      0.50    24.1+/-0.8    20.9+/-0.6  
      1.00    50.6+/-1.7    38.8+/-1.0  
150   0.04    19.4+/-0.4  17.19+/-0.33  
      0.50    30.2+/-0.7    25.3+/-0.5  
      1.00    61.9+/-1.6    45.2+/-0.8  
175   0.04    17.4+/-0.4  15.58+/-0.27  
      0.50    28.1+/-0.6    23.7+/-0.4  
      1.00    57.4+/-1.3    42.4+/-0.7  
200   0.04  15.64+/-0.30  13.96+/-0.22  
      0.50    24.6+/-0.5  20.78+/-0.31  
      1.00    50.5+/-1.0    37.4+/-0.5  
250  -1.00   2.03+/-0.04   1.99+/-0.04  
      0.04  11.08+/-0.25  10.00+/-0.18  
300   0.04   7.99+/-0.23   7.25+/-0.17  
350   0.04   5.98+/-0.23   5.44+/-0.18  
      0.50     9.7+/-0.4   8.36+/-0.25  
      1.00    20.3+/-0.8    15.5+/-0.4  
400  -1.00   1.55+/-0.08   1.49+/-0.07  
      0.04   4.87+/-0.25   4.38+/-0.18  
500   0.04   2.95+/-0.27   2.67+/-0.20  
      0.50     4.9+/-0.4   4.16+/-0.28  
      1.00    10.7+/-1.0     8.0+/-0.4  
700  -1.00   0.66+/-0.11   0.64+/-0.10  
750   0.04   0.70+/-0.13   0.68+/-0.11  
1000 -1.00   0.38+/-0.13   0.37+/-0.11

In [30]:
print(make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2).to_latex())

\begin{tabular}{lllllll}
\toprule
     &       &                  s &                  b &              s/b &     s/sqrt(b) &   s/sqrt(s+b) \\
mass & dbs &                    &                    &                  &               &               \\
\midrule
125  &  0.04 &          1150+/-27 &  (5.31+/-0.19)e+03 &    0.216+/-0.009 &    15.8+/-0.5 &    14.3+/-0.4 \\
     &  0.50 &  (1.76+/-0.05)e+03 &  (5.31+/-0.19)e+03 &    0.331+/-0.015 &    24.1+/-0.8 &    20.9+/-0.6 \\
     &  1.00 &  (3.69+/-0.11)e+03 &  (5.31+/-0.19)e+03 &    0.694+/-0.032 &    50.6+/-1.7 &    38.8+/-1.0 \\
150  &  0.04 &          1370+/-22 &  (4.98+/-0.16)e+03 &    0.275+/-0.010 &    19.4+/-0.4 &  17.19+/-0.33 \\
     &  0.50 &  (2.13+/-0.04)e+03 &  (4.98+/-0.16)e+03 &    0.428+/-0.016 &    30.2+/-0.7 &    25.3+/-0.5 \\
     &  1.00 &  (4.37+/-0.09)e+03 &  (4.98+/-0.16)e+03 &    0.877+/-0.033 &    61.9+/-1.6 &    45.2+/-0.8 \\
175  &  0.04 &          1200+/-16 &  (4.73+/-0.15)e+03 &    0.254+/-0.009 &    17.4+/-0

In [31]:
sample_counts['s'] = sample_counts['SR2'] + sample_counts['SR1']
sample_counts['b'] = sample_counts['SR2dy'] + sample_counts['SR2tt'] + sample_counts['SR1dy'] + sample_counts['SR1tt']
make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2)

s                    b              s/b  \
mass dbs                                                                
125   0.04    (5.89+/-0.06)e+03  (1.362+/-0.033)e+04    0.432+/-0.011   
      0.50    (9.44+/-0.11)e+03  (1.362+/-0.033)e+04    0.693+/-0.018   
      1.00  (2.011+/-0.025)e+04  (1.362+/-0.033)e+04      1.48+/-0.04   
150   0.04    (7.41+/-0.05)e+03  (1.310+/-0.030)e+04    0.566+/-0.013   
      0.50  (1.170+/-0.009)e+04  (1.310+/-0.030)e+04    0.893+/-0.021   
      1.00  (2.471+/-0.021)e+04  (1.310+/-0.030)e+04      1.89+/-0.05   
175   0.04    (6.14+/-0.04)e+03  (1.136+/-0.026)e+04    0.540+/-0.013   
      0.50    (9.93+/-0.07)e+03  (1.136+/-0.026)e+04    0.874+/-0.021   
      1.00  (2.101+/-0.015)e+04  (1.136+/-0.026)e+04      1.85+/-0.04   
200   0.04            4593+/-20    (8.78+/-0.22)e+03    0.523+/-0.013   
      0.50            7386+/-35    (8.78+/-0.22)e+03    0.841+/-0.021   
      1.00  (1.572+/-0.008)e+04    (8.78+/-0.22)e+03      1.79+/-0.05   
250  -1.00          264.3+/-0.9    (5.11+/-0.16)e+03  0.0517+/-0.0016   
      0.04             2357+/-9    (5.11+/-0.16)e+03    0.461+/-0.014   
300   0.04             1221+/-5    (2.99+/-0.12)e+03    0.409+/-0.017   
350   0.04          663.2+/-2.5    (1.77+/-0.10)e+03    0.374+/-0.021   
      0.50             1098+/-5    (1.77+/-0.10)e+03    0.620+/-0.035   
      1.00            2350+/-11    (1.77+/-0.10)e+03      1.33+/-0.07   
400  -1.00         78.04+/-0.26      (9.4+/-0.7)e+02    0.083+/-0.006   
      0.04          369.8+/-1.5      (9.4+/-0.7)e+02    0.393+/-0.030   
500   0.04          134.8+/-0.5      (5.0+/-0.6)e+02    0.271+/-0.033   
      0.50          227.2+/-0.9      (5.0+/-0.6)e+02      0.46+/-0.05   
      1.00          502.0+/-2.4      (5.0+/-0.6)e+02      1.01+/-0.12   
700  -1.00         13.50+/-0.05             136+/-34    0.100+/-0.025   
750   0.04         17.05+/-0.10      (1.4+/-0.4)e+02    0.125+/-0.033   
1000 -1.00        3.631+/-0.014              22+/-15      0.16+/-0.11   

              s/sqrt(b)   s/sqrt(s+b)  
mass dbs                               
125   0.04   50.5+/-0.8    42.2+/-0.5  
      0.50   80.9+/-1.3    62.2+/-0.7  
      1.00  172.3+/-3.0   109.5+/-1.1  
150   0.04   64.7+/-0.9    51.7+/-0.5  
      0.50  102.2+/-1.4    74.3+/-0.6  
      1.00  215.9+/-3.0   127.1+/-0.9  
175   0.04   57.6+/-0.7    46.4+/-0.4  
      0.50   93.2+/-1.2    68.1+/-0.5  
      1.00  197.1+/-2.7   116.8+/-0.7  
200   0.04   49.0+/-0.6  39.71+/-0.35  
      0.50   78.8+/-1.0    58.1+/-0.4  
      1.00  167.7+/-2.2   100.4+/-0.6  
250  -1.00  3.70+/-0.06   3.60+/-0.05  
      0.04   33.0+/-0.5  27.27+/-0.30  
300   0.04   22.3+/-0.5  18.83+/-0.28  
350   0.04   15.8+/-0.4  13.44+/-0.27  
      0.50   26.1+/-0.7    20.5+/-0.4  
      1.00   55.8+/-1.6    36.6+/-0.5  
400  -1.00  2.55+/-0.10   2.45+/-0.09  
      0.04   12.1+/-0.5  10.22+/-0.28  
500   0.04    6.0+/-0.4   5.36+/-0.25  
      0.50   10.2+/-0.6   8.44+/-0.35  
      1.00   22.5+/-1.4    15.9+/-0.5  
700  -1.00  1.16+/-0.15   1.11+/-0.13  
750   0.04  1.46+/-0.19   1.38+/-0.16  
1000 -1.00  0.77+/-0.25   0.72+/-0.20

In [32]:
print(make_s_b(sample_counts)[['s','b','s/b', 's/sqrt(b)', 's/sqrt(s+b)']].round(2).to_latex())

\begin{tabular}{lllllll}
\toprule
     &       &                    s &                    b &              s/b &    s/sqrt(b) &   s/sqrt(s+b) \\
mass & dbs &                      &                      &                  &              &               \\
\midrule
125  &  0.04 &    (5.89+/-0.06)e+03 &  (1.362+/-0.033)e+04 &    0.432+/-0.011 &   50.5+/-0.8 &    42.2+/-0.5 \\
     &  0.50 &    (9.44+/-0.11)e+03 &  (1.362+/-0.033)e+04 &    0.693+/-0.018 &   80.9+/-1.3 &    62.2+/-0.7 \\
     &  1.00 &  (2.011+/-0.025)e+04 &  (1.362+/-0.033)e+04 &      1.48+/-0.04 &  172.3+/-3.0 &   109.5+/-1.1 \\
150  &  0.04 &    (7.41+/-0.05)e+03 &  (1.310+/-0.030)e+04 &    0.566+/-0.013 &   64.7+/-0.9 &    51.7+/-0.5 \\
     &  0.50 &  (1.170+/-0.009)e+04 &  (1.310+/-0.030)e+04 &    0.893+/-0.021 &  102.2+/-1.4 &    74.3+/-0.6 \\
     &  1.00 &  (2.471+/-0.021)e+04 &  (1.310+/-0.030)e+04 &      1.89+/-0.05 &  215.9+/-3.0 &   127.1+/-0.9 \\
175  &  0.04 &    (6.14+/-0.04)e+03 &  (1.136+/-0.026)e+04 &   